#### 1. 단어 표현
- 어떻게 자연어를 컴퓨터에게 인식시킬 수 있을까 ?
- 원 핫 인코딩: 각 단어의 인덱스를 정한 후 각 단어의 벡터에서 그 단어에 해당하는 인덱스의 값을 1로 표현
    - 단점1: 수십만, 수백만개가 넘는 단어를 표현할 때
    - 단점2: 단어가 뭔지만 알려줄 수 있음. 단어의 의미나 특징 같은 것들이 전혀 표현되지 않음
- 카운트 기반 방법: 어떤 글의 문맥 안에 단어가 동시에 등장하는 횟수를 세는 방법
- 예측 방법: 신경망 구조 혹은 어떠한 모델을 사용해 특정 문맥에서 어떤 단어가 나올지를 예측하면서 단어를 벡터로 만드는 방식
    - Word2Vec
        - CBOW: 어떤 단어를 문맥 안의 주변 단어들을 통해 예측하는 방법(창욱은 냉장고에서 _____ 꺼내서 먹었다.)
        - Skip-Gram: 어떤 단어를 가지고 특정 문맥 안의 주변 단어들을 예측하는 방법(_____ _____ 음식을 _____ _____)

#### 2. 텍스트 분류
- 특정 텍스트를 사람들이 정한 몇가지 범주(Class) 중 어느 범주에 속하는지 분류하는 문제
- 지도 학습을 통한 텍스트 분류
    - 데이터에 대해 각각 속한 범주에 대한 값이 이미 주어져 있음
    - 주어진 범주로 학습한 후 결과를 이용해 새로운 글의 범주 예측
- 비지도 학습을 통한 텍스트 분류
    - 데이터만 존재하고, 범주로 나누어져 있지 않음
- 데이터에 정답 라벨이 있느냐 없느냐에 따라 지도 혹은 비지도 학습 선택

#### 3. 텍스트 유사도
- 다른 구조의 문장이지만 의미는 비슷하기 때문에 두 문장의 유사도가 높다고 판단
- 최대한 정량화 해서 모델을 만드는 것이 중요


In [4]:
# TF-IDF를 통한 벡터화 진행
from sklearn.feature_extraction.text import TfidfVectorizer
sent = ('휴일인 오늘도 서쪽을 중심으로 폭염이 이어졌는데요, 내일은 반가운 비 소식이 있습니다.', '폭염을 피해서 휴일에 놀러왔다가 갑작스런 비로 인해 망연자실 하고 있습니다.')
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sent) # 문장 벡터화

idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names_out(), idf)))

{'갑작스런': 1.4054651081081644, '내일은': 1.4054651081081644, '놀러왔다가': 1.4054651081081644, '망연자실': 1.4054651081081644, '반가운': 1.4054651081081644, '비로': 1.4054651081081644, '서쪽을': 1.4054651081081644, '소식이': 1.4054651081081644, '오늘도': 1.4054651081081644, '이어졌는데요': 1.4054651081081644, '인해': 1.4054651081081644, '있습니다': 1.0, '중심으로': 1.4054651081081644, '폭염을': 1.4054651081081644, '폭염이': 1.4054651081081644, '피해서': 1.4054651081081644, '하고': 1.4054651081081644, '휴일에': 1.4054651081081644, '휴일인': 1.4054651081081644}
